# Project 2 - Identification of DFAs by Consistent Reduction


## Imports

In [10]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent / "src"))
Path("../outputs").mkdir(parents=True, exist_ok=True)
from dfa_utils import random_dfa, concurrent_composition
from dfa_examples import dfa_simple
from generate_sets import derive_sets
from reduction import build_initial_cover
from tree_builder import build_prefix_tree, PrefixTreeNode
from Lib import (
    export_dfa_dot, dot_to_png,
    reduce_negative_set,
    splitter, split, update2, char, compute_cover,
    check_cover_redundancy, check_dynamic_consistency, print_cover,
    build_reduced_dfa_from_dynamic_cover
)



## Pipeline 

1) Generation of a reference DFA H through the method dfa_simple().
2) Some utils used to export the obtained DFA in .dot form and, then, to .png

In [11]:
dfa = dfa_simple()
export_dfa_dot(dfa, "../outputs/dfa.dot")   
dot_to_png("../outputs/dfa.dot", "../outputs/dfa.png")

[reduction] .dot saved in ../outputs/dfa.dot
PNG generated: ../outputs/dfa.png


1) Generation of 2 random DFA e to the concurrent composition.
2) Some utils used to export the obtained DFA in .dot form and, then, to .png

In [ ]:
dfa1 = random_dfa(n_states=4, alphabet={'a', 'b', 'c', 'd', 'e'}, seed=42)
print("Stato iniziale:", dfa1.q0)
export_dfa_dot(dfa1, "../outputs/dfa1.dot")   
dot_to_png("../outputs/dfa1.dot", "../outputs/dfa1.png")

dfa2 = random_dfa(n_states=5, alphabet={'a', 'b','c', 'd', 'e'}, seed=53)
print("Stato iniziale:", dfa2.q0)
export_dfa_dot(dfa2, "../outputs/dfa2.dot")   
dot_to_png("../outputs/dfa2.dot", "../outputs/dfa2.png")

composed_dfa = concurrent_composition(dfa1, dfa2)
print("Stato iniziale del DFA composto:", composed_dfa.q0)
export_dfa_dot(composed_dfa, "../outputs/composed_dfa.dot")
dot_to_png("../outputs/composed_dfa.dot", "../outputs/composed_dfa.png")


Stato iniziale: r0
[reduction] .dot saved in ../outputs/dfa1.dot
PNG generated: ../outputs/dfa1.png
Stato iniziale: r4
[reduction] .dot saved in ../outputs/dfa2.dot
PNG generated: ../outputs/dfa2.png
[reduction] .dot saved in ../outputs/composed_dfa.dot
PNG generated: ../outputs/composed_dfa.png


### Select the DFA that you want to reduce

In [13]:
#dfa=dfa #use the reference DFA

#or

dfa = composed_dfa  #use the composed DFA by 2 random DFAs

1) Generation of the three strings sets A, G and N from the previously generated DFA, with a maximum length (of the strings that belongs to each set) of 6
2) Reduction of the set N through the method reduce_negative_set()

In [14]:
A, G, N = derive_sets(dfa, max_len=4)  
print("A =", sorted(A, key=len))
print("G =", sorted(G, key=len))
N=reduce_negative_set(N)
print("N=", N)


Calculating total word count...
Total words to process: 780
A = ['bc', 'beae', 'bead']
G = ['b', 'be', 'ba', 'bee', 'bea', 'beaa', 'beab']
N= {'bad', 'beec', 'bca', 'c', 'beea', 'bec', 'beee', 'bab', 'a', 'bcd', 'd', 'bed', 'e', 'beeb', 'beb', 'beed', 'bd', 'bcb', 'baa', 'bae', 'bac', 'bb', 'bcc', 'bce', 'beac'}


1) Construction of the prefix tre acceptor given the three sets A, G and N and the alphabet of the DFA H to reduce

In [15]:
tree = build_prefix_tree(A, G, N, alphabet=dfa.Sigma)
tree.export_dot("../outputs/prefix_tree2.dot")   
dot_to_png("../outputs/prefix_tree2.dot", "../outputs/prefix_tree2.png")

set()
set()
[tree_builder] saved .dot in ../outputs/prefix_tree2.dot
PNG generated: ../outputs/prefix_tree2.png


### Reduction process

1) Cover 0 represents the initial cover given the prefix tree acceptor. This cover X is non redundant
2) Compute the refined cover through the method compute_cover that generate from the initial non-redundant cover a (H-R_C)-consistent cover (a dynamically consistent cover) that is from definition non redundant

In [16]:
cover0 = build_initial_cover(tree)
print("Initial cover:")
print_cover(cover0)
cover_start=cover0.copy()
cover_final = compute_cover(tree, cover_start)
print("Final cover:")
print_cover(cover_final)

Initial cover:
Cover[1] = {, bc, bead, beae}
Cover[2] = {, b, ba, be, bea, beaa, beab, bee}
Cover[3] = {, a, baa, bab, bac, bad, bae, bb, bca, bcb, bcc, bcd, bce, bd, beac, beb, bec, bed, beea, beeb, beec, beed, beee, c, d, e}
Final cover:
Cover[1] = {be, beaa, beab}
Cover[2] = {ba, beaa, beab, bee}
Cover[3] = {bea, beaa, beab}
Cover[4] = {b, beaa, beab}
Cover[5] = {, a, baa, bab, bac, bad, bae, bb, bca, bcb, bcc, bcd, bce, bd, beac, beb, bec, bed, beea, beeb, beec, beed, beee, c, d, e}
Cover[6] = {bc, bead, beae}


In [17]:
#Debug delle cover finali
check, problemi = check_dynamic_consistency(cover_final, tree.alphabet)
print("\n--- STAMPA DELLA COVER FINALE ---")
print_cover(cover_final)

print("\n--- VERIFICA DELLA DYNAMIC CONSISTENCY ---")
print(f"La cover è dinamically consistent? {check}")
if not check:
    print("\nProblemi trovati:")
    for pi, sigma, motivo in problemi:
        parole = sorted([n.word for n in pi])
        print(f"Pi: {{{', '.join(parole)}}}, evento: {sigma}, motivo: {motivo}")

is_ok, problemi = check_cover_redundancy(cover_final)
if is_ok:
    print("La cover non ha ridondanze!")
else:
    print("Cover ridondante! Problemi trovati:")
    for i, j in problemi:
        print(f"La cella {j+1} è contenuta in cella {i+1}")



--- STAMPA DELLA COVER FINALE ---
Cover[1] = {be, beaa, beab}
Cover[2] = {ba, beaa, beab, bee}
Cover[3] = {bea, beaa, beab}
Cover[4] = {b, beaa, beab}
Cover[5] = {, a, baa, bab, bac, bad, bae, bb, bca, bcb, bcc, bcd, bce, bd, beac, beb, bec, bed, beea, beeb, beec, beed, beee, c, d, e}
Cover[6] = {bc, bead, beae}

--- VERIFICA DELLA DYNAMIC CONSISTENCY ---
La cover è dinamically consistent? True
La cover non ha ridondanze!


### Uso del DFA ridotto


In [18]:
dfa_red = build_reduced_dfa_from_dynamic_cover(tree, cover_final)
export_dfa_dot(dfa_red, "../outputs/reduced_dfa.dot")   
dot_to_png("../outputs/reduced_dfa.dot", "../outputs/reduced_dfa.png")


Initial is  q4
[reduction] .dot saved in ../outputs/reduced_dfa.dot
PNG generated: ../outputs/reduced_dfa.png
